## Work
1. 請改變 reduce_lr 的 patience 和 factor 並比較不同設定下，對訓練/驗證集的影響
2. 請將 optimizer 換成 Adam、RMSprop 搭配 reduce_lr 並比較訓練結果

In [3]:
import os
import keras

# Disable GPU
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [4]:
train, test = keras.datasets.cifar10.load_data()

In [5]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [6]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [8]:
from keras.layers import BatchNormalization

def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [9]:
LEARNING_RATE = 1e-3
EPOCHS = 25 # IF you feel too run to finish, try to make it smaller
BATCH_SIZE = 1024 

In [10]:
# 載入 Callbacks
from keras.callbacks import ReduceLROnPlateau

optimizer_set = [keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=0.95),
                 keras.optimizers.Adam(lr=LEARNING_RATE),
                 keras.optimizers.RMSprop(lr=LEARNING_RATE)]

"""Code Here
建立實驗的比較組合
"""
reduce_lr_factor = [1,2,3]
redice_lr_patient = [1,2,3]

In [ ]:
import itertools
results = {}
for i, (optim, reduce_factor, reduce_patient) in enumerate(itertools.product(optimizer_set, reduce_lr_factor, redice_lr_patient)):
    print("Numbers of exp: %i, reduce_factor: %.2f, reduce_patient: %i" % (i, reduce_factor, reduce_patient))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optim)
    
    """Code Here
    設定 reduce learning rate 的 callback function
    """
    reduce_lr = ReduceLROnPlateau(factor=0.5, 
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience=5, 
                              verbose=1)
    
    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,
              callbacks=[reduce_lr]
             )

    # Collect results
    exp_name_tag = ("exp-%s" % (i))
    results[exp_name_tag] = {'train-loss': model.history.history["loss"],
                             'valid-loss': model.history.history["val_loss"],
                             'train-acc': model.history.history["acc"],
                             'valid-acc': model.history.history["val_acc"]}

Numbers of exp: 0, reduce_factor: 1.00, reduce_patient: 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalizatio

50000/50000 [==============================] - 2s 42us/step - loss: 1.3606 - acc: 0.5282 - val_loss: 1.5471 - val_acc: 0.4579
Epoch 14/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.3406 - acc: 0.5350 - val_loss: 1.5490 - val_acc: 0.4593
Epoch 15/25
50000/50000 [==============================] - 2s 42us/step - loss: 1.3215 - acc: 0.5413 - val_loss: 1.5325 - val_acc: 0.4625
Epoch 16/25
50000/50000 [==============================] - 2s 40us/step - loss: 1.3043 - acc: 0.5476 - val_loss: 1.5167 - val_acc: 0.4701
Epoch 17/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.2857 - acc: 0.5540 - val_loss: 1.5107 - val_acc: 0.4726
Epoch 18/25
50000/50000 [==============================] - 2s 42us/step - loss: 1.2698 - acc: 0.5604 - val_loss: 1.5082 - val_acc: 0.4714
Epoch 19/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.2529 - acc: 0.5675 - val_loss: 1.5105 - val_acc: 0.4735
Epoch 20/25
50000/50000 [=====================

50000/50000 [==============================] - 2s 49us/step - loss: 2.3405 - acc: 0.2416 - val_loss: 2.2369 - val_acc: 0.2994
Epoch 2/25
50000/50000 [==============================] - 2s 40us/step - loss: 1.8693 - acc: 0.3574 - val_loss: 1.9803 - val_acc: 0.3515
Epoch 3/25
50000/50000 [==============================] - 2s 42us/step - loss: 1.7302 - acc: 0.3972 - val_loss: 1.7817 - val_acc: 0.3840
Epoch 4/25
50000/50000 [==============================] - 2s 40us/step - loss: 1.6523 - acc: 0.4219 - val_loss: 1.7133 - val_acc: 0.4031
Epoch 5/25
50000/50000 [==============================] - 2s 40us/step - loss: 1.5987 - acc: 0.4426 - val_loss: 1.6672 - val_acc: 0.4193
Epoch 6/25
50000/50000 [==============================] - 2s 40us/step - loss: 1.5559 - acc: 0.4578 - val_loss: 1.6662 - val_acc: 0.4151
Epoch 7/25
50000/50000 [==============================] - 2s 40us/step - loss: 1.5196 - acc: 0.4701 - val_loss: 1.6317 - val_acc: 0.4266
Epoch 8/25
50000/50000 [============================

50000/50000 [==============================] - 2s 40us/step - loss: 1.1766 - acc: 0.5970 - val_loss: 1.4752 - val_acc: 0.4835
Epoch 25/25
50000/50000 [==============================] - 2s 40us/step - loss: 1.1638 - acc: 0.6003 - val_loss: 1.4781 - val_acc: 0.4870
Numbers of exp: 5, reduce_factor: 2.00, reduce_patient: 3
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_16 (Batc (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalizatio

Epoch 12/25
50000/50000 [==============================] - 2s 40us/step - loss: 1.3673 - acc: 0.5261 - val_loss: 1.5459 - val_acc: 0.4571
Epoch 13/25
50000/50000 [==============================] - 2s 40us/step - loss: 1.3458 - acc: 0.5331 - val_loss: 1.5364 - val_acc: 0.4573
Epoch 14/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.3267 - acc: 0.5409 - val_loss: 1.5273 - val_acc: 0.4568
Epoch 15/25
50000/50000 [==============================] - 2s 42us/step - loss: 1.3084 - acc: 0.5482 - val_loss: 1.5234 - val_acc: 0.4619
Epoch 16/25
50000/50000 [==============================] - 2s 40us/step - loss: 1.2895 - acc: 0.5544 - val_loss: 1.5158 - val_acc: 0.4646
Epoch 17/25
50000/50000 [==============================] - 2s 42us/step - loss: 1.2719 - acc: 0.5610 - val_loss: 1.5095 - val_acc: 0.4706
Epoch 18/25
50000/50000 [==============================] - 2s 46us/step - loss: 1.2542 - acc: 0.5684 - val_loss: 1.5070 - val_acc: 0.4707
Epoch 19/25
50000/50000 [=========

...

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 3s 69us/step - loss: 1.8240 - acc: 0.3701 - val_loss: 2.1279 - val_acc: 0.2944
Epoch 2/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.5401 - acc: 0.4579 - val_loss: 3.0605 - val_acc: 0.2019
Epoch 3/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.4375 - acc: 0.4941 - val_loss: 2.1108 - val_acc: 0.3047
Epoch 4/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.3558 - acc: 0.5181 - val_loss: 2.0225 - val_acc: 0.3282
Epoch 5/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.2945 - acc: 0.5413 - val_loss: 2.0636 - val_acc: 0.3371
Epoch 6/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.2467 - acc: 0.5593 - val_loss: 1.8002 - val_acc: 0.3863
Epoch 7/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.1952 - acc: 0.5794 - val_loss: 1.9832 - val_acc

Epoch 22/25
50000/50000 [==============================] - 2s 41us/step - loss: 0.7990 - acc: 0.7425 - val_loss: 1.4457 - val_acc: 0.5058
Epoch 23/25
50000/50000 [==============================] - 2s 41us/step - loss: 0.7807 - acc: 0.7519 - val_loss: 1.4601 - val_acc: 0.4996
Epoch 24/25
50000/50000 [==============================] - 2s 41us/step - loss: 0.7646 - acc: 0.7574 - val_loss: 1.4425 - val_acc: 0.5051

Epoch 00024: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
Epoch 25/25
50000/50000 [==============================] - 2s 41us/step - loss: 0.7228 - acc: 0.7776 - val_loss: 1.4248 - val_acc: 0.5113
Numbers of exp: 20, reduce_factor: 1.00, reduce_patient: 3
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)    

50000/50000 [==============================] - 2s 42us/step - loss: 1.4124 - acc: 0.5099 - val_loss: 1.5906 - val_acc: 0.4462
Epoch 10/25
50000/50000 [==============================] - 2s 42us/step - loss: 1.3821 - acc: 0.5208 - val_loss: 1.5765 - val_acc: 0.4494
Epoch 11/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.3530 - acc: 0.5326 - val_loss: 1.5691 - val_acc: 0.4497
Epoch 12/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.3295 - acc: 0.5413 - val_loss: 1.5616 - val_acc: 0.4531
Epoch 13/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.3031 - acc: 0.5507 - val_loss: 1.5516 - val_acc: 0.4567
Epoch 14/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.2812 - acc: 0.5598 - val_loss: 1.5498 - val_acc: 0.4575
Epoch 15/25
50000/50000 [==============================] - 2s 41us/step - loss: 1.2590 - acc: 0.5678 - val_loss: 1.5410 - val_acc: 0.4605
Epoch 16/25
50000/50000 [=====================

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 4s 76us/step - loss: 2.2144 - acc: 0.2706 - val_loss: 2.0652 - val_acc: 0.3163
Epoch 2/25
50000/50000 [==============================] - 2s 42us/step - loss: 1.8506 - acc: 0.3642 - val_loss: 1.9096 - val_acc: 0.3540
Epoch 3/25
50000/50000 [==============================] - 2s 42us/step - loss: 1.7131 - acc: 0.4050 - val_loss: 1.7855 - val_acc: 0.3893
Epoch 4/25
50000/50000 [==============================] - 2s 42us/step - loss: 1.6275 - acc: 0.4324 - val_loss: 1.7238 - val_acc: 0.4053
Epoch 5/25
50000/50000 [==============================] - 2s 42us/step - loss: 1.5673 - acc: 0.4538 - val_loss: 1.6881 - val_acc: 0.4148
Epoch 6/25
50000/50000 [==============================] - 2s 42us/step - loss: 1.5164 - acc: 0.4738 - val_loss: 1.6598 - val_acc: 0.4226
Epoch 7/25
50000/50000 [==============================] - 2s 42us/step - loss: 1.4745 - acc: 0.4875 - val_loss: 1.6162 - val_acc

50000/50000 [==============================] - 2s 41us/step - loss: 1.0794 - acc: 0.6379 - val_loss: 1.4847 - val_acc: 0.4824
Epoch 25/25
50000/50000 [==============================] - 2s 42us/step - loss: 1.0632 - acc: 0.6434 - val_loss: 1.4974 - val_acc: 0.4732
Numbers of exp: 25, reduce_factor: 3.00, reduce_patient: 2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_76 (Batc (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalizati

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
%matplotlib inline
NUM_COLORS = len(results.keys())

cm = plt.get_cmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS-1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
color_bar = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()